In [1]:
%%writefile camera.py
import time

import cv2

pic_num = 30
root = ("../face_pic/")
bool_break = 0
#웹캠에서 영상을 읽어온다
cap = cv2.VideoCapture(0)
cap.set(3, 640) #WIDTH
cap.set(4, 480) #HEIGHT

#얼굴 인식 캐스케이드 파일 읽는다
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
i = 0
while(True):
    # frame 별로 capture 한다
    ret, frame = cap.read()
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    frame_c = frame.copy()
    # 인식된 얼굴에 사각형을 출력한다
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x-40,y-40),(x+w+40,y+h+40),(255,0,0),2)
        # cropping & save
        face = frame_c[y-30:y+h+30,x-30:x+w+30]
        cv2.imwrite(root+str(i)+".jpg",face)
        i += 1
        time.sleep(0.5)
        if i == pic_num:
            bool_break = 1
            break        
    cv2.imshow('frame',frame)
    if bool_break == 1:
        break
    
    if cv2.waitKey(100) & 0xFF == ord('q'):               
        break
    
cap.release()
cv2.destroyAllWindows()


In [2]:
from keras import models
from keras import layers
from keras.applications import VGG16

In [7]:
# conv_base = VGG16(weights = 'imagenet',
#                  include_top = False,
#                  input_shape =(200,200,3))
face_recogn = models.Sequential()
face_recogn.add(layers.Conv2D(16,(3,3),activation='relu',input_shape=(200,200,3)))
face_recogn.add(layers.MaxPooling2D((2,2)))
face_recogn.add(layers.Conv2D(32,(3,3),activation='relu'))
face_recogn.add(layers.MaxPooling2D((2,2)))                
face_recogn.add(layers.Conv2D(64,(3,3),activation='relu'))
face_recogn.add(layers.MaxPooling2D((2,2)))                
# face_recogn.add(conv_base)
face_recogn.add(layers.Flatten())
face_recogn.add(layers.Dense(64,activation='relu'))
face_recogn.add(layers.Dense(32,activation='relu'))
face_recogn.add(layers.Dense(2,activation='softmax'))

face_recogn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 198, 198, 16)      448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 97, 97, 32)        4640      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 33856)            